                                OVERALL AIM OF THE PROJECT
The overall model aims to analyze video data by leveraging spatiotemporal features extracted using the I3D model, and then classifying the videos based on those features into categories such as "peace" or "brawl." Additionally, by integrating YOLOv3-based people detection, the system can provide further insights—such as counting the number of people present—which might be indicative of the intensity of a brawl.

The code imports necessary libraries (OpenCV, NumPy, TensorFlow, and TensorFlow Hub) and sets up some parameters like the dataset path, video categories, and input size/frame count expected by the I3D model. It loads the I3D (Inflated 3D ConvNet) model from TensorFlow Hub. This model is pre-trained on the Kinetics-400 dataset and is used for extracting spatiotemporal features from videos.

The code iterates over two video categories ("peace" and "brawl"). For each video, it extracts frames using the helper function, normalizes them, and passes them through the I3D model to extract features.

In [61]:
import os
import glob
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Parameters
DATASET_PATH = r"C:\Users\Nitro 5\PycharmProjects\pythonProject1\Dataset"
CATEGORIES = ["peace", "brawl"]   # Folder names; 0: peace, 1: brawl
NUM_I3D_FRAMES = 64               # I3D expects 64 frames per video
FRAME_SIZE = (224, 224)           # I3D default input size

# Explicitly set signature to "default" so that we expect the default behavior.
i3d_url = "https://tfhub.dev/deepmind/i3d-kinetics-400/1"
i3d_layer = hub.KerasLayer(i3d_url, trainable=False, signature="default")

def load_video_frames_i3d(video_path, num_frames=NUM_I3D_FRAMES, size=FRAME_SIZE):
    """
    Loads a video, samples or pads to `num_frames` frames, resizes each frame to `size`,
    and returns an array of frames.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0:
        cap.release()
        return None

    # Uniform sampling indices (if video has fewer frames, use all available indices)
    if total_frames >= num_frames:
        frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    else:
        frame_indices = np.arange(total_frames)

    frames = []
    frame_id = 0
    ret = True
    while ret:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_id in frame_indices:
            frame = cv2.resize(frame, size)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
        frame_id += 1
    cap.release()

    # Pad with the last frame if fewer than num_frames
    if len(frames) < num_frames:
        last_frame = frames[-1] if frames else np.zeros((size[1], size[0], 3), dtype=np.uint8)
        for _ in range(num_frames - len(frames)):
            frames.append(last_frame)
    
    return np.array(frames)

# Lists to store extracted features and corresponding labels
video_features = []
video_labels = []

for label, category in enumerate(CATEGORIES):
    folder_path = os.path.join(DATASET_PATH, category)
    video_files = glob.glob(os.path.join(folder_path, "*.mp4"))  # Adjust extension if needed
    print(f"Processing {len(video_files)} videos in category '{category}'...")
    for video_path in video_files:
        frames = load_video_frames_i3d(video_path)
        if frames is not None:
            # Normalize frames to [0,1]
            frames = frames.astype('float32') / 255.0
            # Expand dims to create a batch of size 1: shape becomes (1, 64, 224, 224, 3)
            video_tensor = np.expand_dims(frames, axis=0)
            # Extract features using the I3D model (returns a tensor directly)
            features_tensor = i3d_layer(video_tensor)
            # Convert tensor to numpy array and remove batch dimension
            features = np.squeeze(features_tensor.numpy())
            video_features.append(features)
            video_labels.append(label)
        else:
            print(f"Warning: Could not load video: {video_path}")

video_features = np.array(video_features)
video_labels = np.array(video_labels)

print("Extracted features shape:", video_features.shape)
print("Labels shape:", video_labels.shape)

# Save the features and labels for later training
np.save("i3d_video_features.npy", video_features)
np.save("i3d_video_labels.npy", video_labels)
print("I3D feature extraction complete and data saved.")


Processing 800 videos in category 'peace'...
Processing 800 videos in category 'brawl'...
Extracted features shape: (1600, 400)
Labels shape: (1600,)
I3D feature extraction complete and data saved.


This code loads pre-extracted I3D video features and their corresponding labels, then builds, trains, and evaluates a simple Multi-Layer Perceptron (MLP) classifier to distinguish between two classes ("peace" vs. "brawl"). It uses an 80/20 train-test split-ratio , employs callbacks for early stopping and model checkpointing, and finally computes evaluation metrics such as accuracy, F1 score, confusion matrix, and a detailed classification report before saving the final model.

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# Load the extracted I3D features and labels
features = np.load("i3d_video_features.npy")  # shape: (num_videos, feature_dim)
labels = np.load("i3d_video_labels.npy")        # shape: (num_videos,)

print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

# Split data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels
)

# Build a simple MLP classifier on top of the I3D features
model = Sequential([
    Dense(256, activation='relu', input_shape=(features.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (peace:0, brawl:1)
])

# Compile the model with Adam optimizer and binary crossentropy loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# Define callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_i3d_classifier.h5", monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the classifier
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

# Evaluate the classifier on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Generate predictions on the test set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Calculate F1 score and print evaluation metrics
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))

# Save the final trained model
model.save("final_i3d_classifier.h5")
print("Model training complete and model saved.")


Features shape: (1600, 400)
Labels shape: (1600,)


C:\Users\Nitro 5\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 256)            │       102,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,681 (530.00 KB)

...

72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9066 - loss: 0.2593 - val_accuracy: 0.9531 - val_loss: 0.1009
Epoch 14/50
40/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9119 - loss: 0.2070 
Epoch 14: val_loss did not improve from 0.10090
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9129 - loss: 0.2105 - val_accuracy: 0.9531 - val_loss: 0.1032
Epoch 15/50
42/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9049 - loss: 0.2638 
Epoch 15: val_loss improved from 0.10090 to 0.09716, saving model to best_i3d_classifier.h5


72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9140 - loss: 0.2365 - val_accuracy: 0.9531 - val_loss: 0.0972
Epoch 16/50
41/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9226 - loss: 0.2184 
Epoch 16: val_loss improved from 0.09716 to 0.09582, saving model to best_i3d_classifier.h5


72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9273 - loss: 0.2003 - val_accuracy: 0.9609 - val_loss: 0.0958
Epoch 17/50
39/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9196 - loss: 0.2191 
Epoch 17: val_loss improved from 0.09582 to 0.08521, saving model to best_i3d_classifier.h5


72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9236 - loss: 0.2027 - val_accuracy: 0.9688 - val_loss: 0.0852
Epoch 18/50
36/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9412 - loss: 0.1478 
Epoch 18: val_loss did not improve from 0.08521
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9374 - loss: 0.1492 - val_accuracy: 0.9609 - val_loss: 0.0869
Epoch 19/50
37/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9437 - loss: 0.1957 
Epoch 19: val_loss did not improve from 0.08521
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9422 - loss: 0.1845 - val_accuracy: 0.9688 - val_loss: 0.0852
Epoch 20/50
42/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9625 - loss: 0.0878 
Epoch 20: val_loss did not improve from 0.08521
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9571 - loss: 0.1029 - val_accuracy: 0.9609 - val_loss: 0.0911
Epoch 21/50
41/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9523 - loss: 0.1777 
Epoch 21: val_loss did not improve from 0.08521
72/7

F1 Score: 0.9345
Confusion Matrix:
[[141  19]
 [  3 157]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.88      0.93       160
           1       0.89      0.98      0.93       160

    accuracy                           0.93       320
   macro avg       0.94      0.93      0.93       320
weighted avg       0.94      0.93      0.93       320

Model training complete and model saved.


In [52]:
model.save("final_i3d_classifier.keras")
print("Model training complete and model saved.")

Model training complete and model saved.


In [53]:
import os
import glob
import numpy as np
import cv2
import tensorflow as tf
import tensorflow_hub as hub

# Load trained classifier model
model = tf.keras.models.load_model("final_i3d_classifier.keras")

# Load I3D feature extractor
i3d_url = "https://tfhub.dev/deepmind/i3d-kinetics-400/1"
i3d_layer = hub.KerasLayer(i3d_url, trainable=False, signature="default")

# Parameters
TEST_DATASET_PATH = r"C:\Users\Nitro 5\PycharmProjects\pythonProject1\Test"
CATEGORIES = ["peace_test", "brawl_test"]  # 0: peace, 1: brawl
NUM_I3D_FRAMES = 64  # I3D expects 64 frames
FRAME_SIZE = (224, 224)  # I3D default input size


C:\Users\Nitro 5\venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [54]:
def load_video_frames_i3d(video_path, num_frames=NUM_I3D_FRAMES, size=FRAME_SIZE):
    """
    Loads a video, samples/pads it to `num_frames`, resizes frames, and returns a NumPy array.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0:
        cap.release()
        return None

    # Sample frames uniformly or use all frames if fewer than num_frames
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int) if total_frames >= num_frames else np.arange(total_frames)

    frames = []
    frame_id = 0
    ret = True
    while ret:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_id in frame_indices:
            frame = cv2.resize(frame, size)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
        frame_id += 1
    cap.release()

    # Pad with last frame if too few
    if len(frames) < num_frames:
        last_frame = frames[-1] if frames else np.zeros((size[1], size[0], 3), dtype=np.uint8)
        for _ in range(num_frames - len(frames)):
            frames.append(last_frame)
    
    return np.array(frames)



In [55]:
# Lists to store predictions and actual labels
test_features = []
test_labels = []
video_paths = []

for label, category in enumerate(CATEGORIES):
    folder_path = os.path.join(TEST_DATASET_PATH, category)
    video_files = glob.glob(os.path.join(folder_path, "*.mp4"))  # Change extension if needed
    print(f"Processing {len(video_files)} videos in '{category}'...")

    for video_path in video_files:
        frames = load_video_frames_i3d(video_path)
        if frames is not None:
            # Normalize frames to [0,1]
            frames = frames.astype('float32') / 255.0
            # Expand dims to create a batch of size 1: (1, 64, 224, 224, 3)
            video_tensor = np.expand_dims(frames, axis=0)
            # Extract I3D features
            features_tensor = i3d_layer(video_tensor)
            # Convert to numpy array and remove batch dimension
            features = np.squeeze(features_tensor.numpy())
            test_features.append(features)
            test_labels.append(label)
            video_paths.append(video_path)
        else:
            print(f"Warning: Could not load video: {video_path}")

# Convert to NumPy arrays
test_features = np.array(test_features)
test_labels = np.array(test_labels)

print("Test features shape:", test_features.shape)
print("Test labels shape:", test_labels.shape)

# Make predictions
pred_probs = model.predict(test_features)
pred_labels = (pred_probs > 0.5).astype(int).flatten()



Processing 200 videos in 'peace_test'...
Processing 200 videos in 'brawl_test'...
Test features shape: (400, 400)
Test labels shape: (400,)
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [63]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Print classification metrics
print("\nConfusion Matrix:")
print(confusion_matrix(test_labels, pred_labels))

print("\nClassification Report:")
print(classification_report(test_labels, pred_labels, target_names=["peace", "brawl"]))

# Calculate and print F1 score
f1 = f1_score(test_labels, pred_labels)
print(f"\nF1 Score: {f1:.4f}")

# Generate video IDs (e.g., Brawl_Test1, Peace_Test2)
video_ids = []
category_counts = {"peace": 0, "brawl": 1}

for i, path in enumerate(video_paths):
    category = "brawl" if test_labels[i] == 1 else "peace"
    video_id = f"{category.capitalize()}_Test{category_counts[category]}"
    video_ids.append(video_id)
    category_counts[category] += 1

# Create DataFrame and save predictions
df = pd.DataFrame({"videoID": video_ids, "prediction": ["peace" if p == 0 else "brawl" for p in pred_labels]})
df.to_csv("predictions.csv", index=False)

print("\nPredictions saved to predictions.csv")



Confusion Matrix:
[[180  20]
 [  2 198]]

Classification Report:
              precision    recall  f1-score   support

       peace       0.99      0.90      0.94       200
       brawl       0.91      0.99      0.95       200

    accuracy                           0.94       400
   macro avg       0.95      0.95      0.94       400
weighted avg       0.95      0.94      0.94       400


F1 Score: 0.9474

Predictions saved to predictions.csv


In [60]:
misclassified = np.where(test_labels != pred_labels)[0]
print(f"\nMisclassified videos ({len(misclassified)}):")
for i in misclassified:
    print(f"Actual: {CATEGORIES[test_labels[i]]}, Predicted: {CATEGORIES[pred_labels[i]]}, File: {video_paths[i]}")


Misclassified videos (22):
Actual: peace, Predicted: brawl, File: C:\Users\Nitro 5\PycharmProjects\pythonProject1\Test\peace_test\PeaceTest_105.mp4
Actual: peace, Predicted: brawl, File: C:\Users\Nitro 5\PycharmProjects\pythonProject1\Test\peace_test\PeaceTest_106.mp4
Actual: peace, Predicted: brawl, File: C:\Users\Nitro 5\PycharmProjects\pythonProject1\Test\peace_test\PeaceTest_107.mp4
Actual: peace, Predicted: brawl, File: C:\Users\Nitro 5\PycharmProjects\pythonProject1\Test\peace_test\PeaceTest_108.mp4
Actual: peace, Predicted: brawl, File: C:\Users\Nitro 5\PycharmProjects\pythonProject1\Test\peace_test\PeaceTest_109.mp4
Actual: peace, Predicted: brawl, File: C:\Users\Nitro 5\PycharmProjects\pythonProject1\Test\peace_test\PeaceTest_110.mp4
Actual: peace, Predicted: brawl, File: C:\Users\Nitro 5\PycharmProjects\pythonProject1\Test\peace_test\PeaceTest_111.mp4
Actual: peace, Predicted: brawl, File: C:\Users\Nitro 5\PycharmProjects\pythonProject1\Test\peace_test\PeaceTest_122.mp4
Actu

In [20]:
!pip install nbformat

In [21]:
!python -m nbformat --validate Brawl_Detector.ipynb


C:\Users\Nitro 5\venv\Scripts\python.exe: No module named nbformat.__main__; 'nbformat' is a package and cannot be directly executed


In [22]:
!jupyter nbconvert --to notebook --inplace Brawl_Detector.ipynb


This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute the notebook prior to export.
    Equivalent to: [--ExecutePr

[NbConvertApp] WARNING | pattern 'your_notebook.ipynb' matched no files
